# Twitch PostgreSQL database

### PostgreSQL connection

Based on https://naysan.ca/2020/05/31/postgresql-to-pandas/

In [1]:
# Librairies
import psycopg2
import pandas as pd
import sys

In [2]:
# Connection parameters to login
co_param = {
    "host"      : "twitch.caampywfg0rz.us-east-1.rds.amazonaws.com",
    "database"  : "Twitch",
    "user"      : "GaTech_team_96",
    "password"  : "i-love-my-coffee-without-milk-and-sugar-at-800AM"
}

In [3]:
def connect(co_param):
    """
    Connect to the PostgreSQL database server
    """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**co_param)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [4]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

### stream_data database

In [5]:
# SQL query
sql_query = """SELECT * FROM stream_data"""

# Column names
stream_data_col_names = ["game_id","stream_id","language","started_at","title",
                            "stream_type","user_id","user_name","viewer_count","user_login","game_name",
                            "thumbnail_url","tag_ids","is_mature","time_logged"]

# Retrieving the data
stream_data = postgresql_to_dataframe(connect(co_param), sql_query, stream_data_col_names)
stream_data.head()

Connecting to the PostgreSQL database...
Connection successful


,game_id,stream_id,language,started_at,title,stream_type,user_id,user_name,viewer_count,user_login,game_name,thumbnail_url,tag_ids,is_mature,time_logged
0,493597,40463396137,pt,2022-11-08T17:18:43Z,[!DROP TIER 1] SUB GRATUITO !GRATIS - !KEY !FR...,live,134233365,Finger,267,finger,New World,https://static-cdn.jtvnw.net/previews-ttv/live...,"{39ee8140-901a-4762-bfca-8260dea1310f,c2542d6d...",True,2022-11-09 12:24:55
1,493597,40052399384,de,2022-11-09T10:53:42Z,[Knights of The Coconut | Marauder] Drops (2/4...,live,41945115,GippiGaming,266,gippigaming,New World,https://static-cdn.jtvnw.net/previews-ttv/live...,"{9166ad14-41f1-4b04-a3b8-c8eb838c6be6,c2542d6d...",False,2022-11-09 12:24:55
2,493597,40052332488,ru,2022-11-09T10:19:35Z,🔴ВРЫВАЕМСЯ НА ФРЕШ СЕРВЕРА! !DROPS Сервер Imho...,live,41237871,SirDaveJones,199,sirdavejones,New World,https://static-cdn.jtvnw.net/previews-ttv/live...,"{0569b171-2a2b-476e-a596-5bdfb45a1327,c2542d6d...",False,2022-11-09 12:24:55
3,493597,46278433612,es,2022-11-09T07:34:28Z,Buen dia de miercoles,live,40113517,GgSanTomE,190,ggsantome,New World,https://static-cdn.jtvnw.net/previews-ttv/live...,"{c2542d6d-cd10-4532-919b-3d19f30a768b,d4bb9c58...",False,2022-11-09 12:24:55
4,493597,41479709947,en,2022-11-09T08:39:01Z,[DROPS] Hatchet/X PvP | Light Melee | !build !...,live,41556524,TheWhelps,178,thewhelps,New World,https://static-cdn.jtvnw.net/previews-ttv/live...,"{6ea6bca4-4712-4ab9-a906-e3336a9d8039,c2542d6d...",False,2022-11-09 12:24:55


###

0. filtering at english game
1. temp => get the diff of time_logged and started_at
2. how is the game behaving after 1week or 2? >>> defining a target for the MLalgo
        input target
        avg/medium/total viewers >>> 
3. Post-temp >>> sentiment analysis >>> title
4. Audience target > is_mature of the game
5. python dict for the categories of the game



In [7]:
temp = pd.to_datetime(stream_data["time_logged"])-pd.to_datetime(stream_data["started_at"])

TypeError: Cannot subtract tz-naive and tz-aware datetime-like objects.

In [8]:
data = stream_data.copy()
data.shift(3).head()

,game_id,stream_id,language,started_at,title,stream_type,user_id,user_name,viewer_count,user_login,game_name,thumbnail_url,tag_ids,is_mature,time_logged
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,493597,40463396137,pt,2022-11-08T17:18:43Z,[!DROP TIER 1] SUB GRATUITO !GRATIS - !KEY !FR...,live,134233365,Finger,267.0,finger,New World,https://static-cdn.jtvnw.net/previews-ttv/live...,"{39ee8140-901a-4762-bfca-8260dea1310f,c2542d6d...",True,2022-11-09 12:24:55
4,493597,40052399384,de,2022-11-09T10:53:42Z,[Knights of The Coconut | Marauder] Drops (2/4...,live,41945115,GippiGaming,266.0,gippigaming,New World,https://static-cdn.jtvnw.net/previews-ttv/live...,"{9166ad14-41f1-4b04-a3b8-c8eb838c6be6,c2542d6d...",False,2022-11-09 12:24:55


In [9]:
stream_data.head()

,game_id,stream_id,language,started_at,title,stream_type,user_id,user_name,viewer_count,user_login,game_name,thumbnail_url,tag_ids,is_mature,time_logged
0,493597,40463396137,pt,2022-11-08T17:18:43Z,[!DROP TIER 1] SUB GRATUITO !GRATIS - !KEY !FR...,live,134233365,Finger,267,finger,New World,https://static-cdn.jtvnw.net/previews-ttv/live...,"{39ee8140-901a-4762-bfca-8260dea1310f,c2542d6d...",True,2022-11-09 12:24:55
1,493597,40052399384,de,2022-11-09T10:53:42Z,[Knights of The Coconut | Marauder] Drops (2/4...,live,41945115,GippiGaming,266,gippigaming,New World,https://static-cdn.jtvnw.net/previews-ttv/live...,"{9166ad14-41f1-4b04-a3b8-c8eb838c6be6,c2542d6d...",False,2022-11-09 12:24:55
2,493597,40052332488,ru,2022-11-09T10:19:35Z,🔴ВРЫВАЕМСЯ НА ФРЕШ СЕРВЕРА! !DROPS Сервер Imho...,live,41237871,SirDaveJones,199,sirdavejones,New World,https://static-cdn.jtvnw.net/previews-ttv/live...,"{0569b171-2a2b-476e-a596-5bdfb45a1327,c2542d6d...",False,2022-11-09 12:24:55
3,493597,46278433612,es,2022-11-09T07:34:28Z,Buen dia de miercoles,live,40113517,GgSanTomE,190,ggsantome,New World,https://static-cdn.jtvnw.net/previews-ttv/live...,"{c2542d6d-cd10-4532-919b-3d19f30a768b,d4bb9c58...",False,2022-11-09 12:24:55
4,493597,41479709947,en,2022-11-09T08:39:01Z,[DROPS] Hatchet/X PvP | Light Melee | !build !...,live,41556524,TheWhelps,178,thewhelps,New World,https://static-cdn.jtvnw.net/previews-ttv/live...,"{6ea6bca4-4712-4ab9-a906-e3336a9d8039,c2542d6d...",False,2022-11-09 12:24:55


### game_info database

In [6]:
# SQL query
sql_query = """SELECT * FROM game_info"""

# Column names
game_info_col_names = ["game_id","game_name","game_picture_url","time_logged"]

# Retrieving the data
game_info = postgresql_to_dataframe(connect(co_param), sql_query, game_info_col_names)
game_info.head()

Connecting to the PostgreSQL database...
Connection successful


,game_id,game_name,game_picture_url,time_logged
0,743,Chess,https://static-cdn.jtvnw.net/ttv-boxart/743-{w...,2022-11-09 12:24:36
1,2748,Magic: The Gathering,https://static-cdn.jtvnw.net/ttv-boxart/2748-{...,2022-11-09 12:24:36
2,11989,StarCraft,https://static-cdn.jtvnw.net/ttv-boxart/11989_...,2022-11-09 12:24:36
3,12839,Heroes of Might and Magic III: The Restoration...,https://static-cdn.jtvnw.net/ttv-boxart/12839_...,2022-11-09 12:24:36
4,12924,Warcraft III,https://static-cdn.jtvnw.net/ttv-boxart/12924-...,2022-11-09 12:24:36
